This notebook demonstrates how `conv1d` works in Keras and is based upon Jussi Huotari's [excellent article](http://www.jussihuotari.com/2017/12/20/spell-out-convolution-1d-in-cnns/).

## 1D Convolution in Numpy

Let's start by manually computing the result of convolving a 1D filter with an input sequence. 

In [1]:
import numpy as np

conv1d_filter = np.array([1,2])
toyX = np.array([0, 3, 4, 5])
print('Conv1d input:', toyX)

result = []
for i in range(3):
    print(toyX[i:i+2], "*", conv1d_filter, "=", toyX[i:i+2] * conv1d_filter)
    result.append(np.sum(toyX[i:i+2] * conv1d_filter))

print("Conv1d output:", result)

Conv1d input: [0 3 4 5]
[0 3] * [1 2] = [0 6]
[3 4] * [1 2] = [3 8]
[4 5] * [1 2] = [ 4 10]
Conv1d output: [6, 11, 14]


Now, suppose we are given the input and output and asked to determine the convolution kernel. One approach is to use the "Convolution Theorem", e.g. as done [here](https://stackoverflow.com/questions/8478276/finding-the-convolution-kernel-in-matlab). 

Another approach is to guess the kernel, estimate how far the corresponding predicted output is from the true output and shift the elements of the kernel in the direction that makes that "loss" smaller. In other words, use gradient descent...

## Computing a 1D convolution filter using Keras

In [5]:
from keras import backend as K
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv1D

K.clear_session()

# input to Keras' conv1d has shape number_examples X length_of_sequence X number_input_channels
toyX = np.array([0, 3, 4, 5]).reshape(1,4,1)

# just re-using the convolutional output in the preceding example
toyY = np.array([6, 11, 14]).reshape(1,3,1)

# https://keras.io/layers/convolutional/#conv1d
# before Keras 2.0 the layers were called ConvolutionalND for N = 1, 2, 3, and since Keras 2.0 they are just called ConvND.
toyModel = Sequential([    
    Conv1D(filters=1, kernel_size=2, strides=1, padding='valid', use_bias=False, input_shape=(4,1), name='c1d')
])

# https://github.com/keras-team/keras/blob/dcb5e3cf152977209c7122f6b07d6c86fb4cf1e6/keras/losses.py#L99
toyModel.compile(optimizer=Adam(lr=5e-2), loss='mae')

# kernel has dimension kernel_size X number_input_channels X number_output_channels
print('shape of conv filter', toyModel.layers[0].get_weights()[0].shape)
print('')

print('conv weights')
print('------------')
for epoch in range(200):
    history = toyModel.fit(toyX, toyY, verbose=0)
    if epoch%20 == 0:
        print("{:3d} {} \t {}".format(epoch, 
                                      toyModel.layers[0].get_weights()[0][:,0,0], 
                                      history.history))

shape of conv filter (2, 1, 1)

conv weights
------------
  0 [-0.72896165  0.5631139 ] 	 {'loss': [10.098454475402832]}
 20 [0.27028498 1.5623608 ] 	 {'loss': [3.7698917388916016]}
 40 [0.9788118 2.2267246] 	 {'loss': [0.9194195866584778]}
 60 [0.98986524 2.041007  ] 	 {'loss': [0.11650880426168442]}
 80 [0.9991782 1.9600047] 	 {'loss': [0.1810016632080078]}
100 [0.99584895 1.9986793 ] 	 {'loss': [0.02970091439783573]}
120 [1.0025234 1.9819707] 	 {'loss': [0.01458613108843565]}
140 [0.9962751 2.0011797] 	 {'loss': [0.008762359619140625]}
160 [0.9995124 1.996057 ] 	 {'loss': [0.031674545258283615]}
180 [1.0029459 1.9817123] 	 {'loss': [0.014955838210880756]}


The convolution weights gravitate towards the expected values. 

Key to the success of this approach is having enough data. Concretely, each position of the filter in the input sequence corresponds to an equation involving all of the components in the filter. To uniquely solve for those components, it is necessary to have at least as many equations as there are components. The number of equations is determined by the length of the input sequence and by the number of those sequences. 

For example, in our case, the filter has 2 components, so it is necessary to have at least 2 equations. There are 3 unique positions in a single example, so there are 3 equations for 2 unknowns, and so the system of equations is overdetermined, and it is likely that there is only one solution, which gradient descent finds easily. 

If we had less data, e.g. the sequence length was shorter, or if the model was more complex, e.g. the filter was longer, then the system of equations would have been under-determined, with many possible solutions, corresponding to local optima in the space of parameters. This type of scenario is called "overfitting". 

## Finding a 1D convolution filter when there is more than one input channel

An input sequence may have more than one "channel". Can gradient descent find the corresponding filter, given an input and output sequence? 

In [6]:
K.clear_session()

toyX = np.array([[0, 0], [3, 6], [4, 7], [5, 8], [1, 2], [4, 5], [2, 0], [0, 1]]).reshape(1,8,2) # single input sequence with 8 steps and 2 channels
toyY = np.array([14, 20, 14, 15, 10, 4]).reshape(1,6,1) # obtained by convolving the input sequence with the filter [[1,0], [0,1], [2,0]]
toyModel = Sequential([
 Conv1D(filters=1, kernel_size=3, strides=1, padding='valid', use_bias=False, input_shape=(8,2), name='c1d')
])
toyModel.compile(optimizer=Adam(lr=5e-2), loss='mae')

print('shape of conv filter', toyModel.layers[0].get_weights()[0].shape)
print('conv weights')
print('------------')
for epoch in range(1000):
    history = toyModel.fit(toyX, toyY, verbose=0)
    if epoch%100 == 0:
        # kernel has dimension: kernel_size X number_input_channels X number_output_channels
        print("{:3d} {} \t {}".format(epoch, 
                                      toyModel.layers[0].get_weights()[0][:,:,0], 
                                      history.history))
print('prediction on training data:', toyModel.predict(toyX))

shape of conv filter (3, 2, 1)
conv weights
------------
  0 [[ 0.00561469 -0.2564376 ]
 [ 0.6226566   0.52996373]
 [-0.7588456   0.62725854]] 	 {'loss': [10.279986381530762]}
100 [[0.2864395  0.4370375 ]
 [0.3900503  0.67185867]
 [0.96525764 0.6868093 ]] 	 {'loss': [0.6920822262763977]}
200 [[0.3153597  0.41625863]
 [0.26372367 0.7739245 ]
 [1.8895061  0.11112271]] 	 {'loss': [0.1427873820066452]}
300 [[0.53008467 0.28652686]
 [0.1805524  0.8582995 ]
 [1.9257807  0.06928745]] 	 {'loss': [0.11246975511312485]}
400 [[0.69915605 0.17858563]
 [0.12399938 0.9194845 ]
 [1.9783874  0.06550355]] 	 {'loss': [0.40661606192588806]}
500 [[0.77650887 0.13201742]
 [0.08307225 0.91956806]
 [1.984738   0.03154069]] 	 {'loss': [0.05280423164367676]}
600 [[0.86430496 0.05792169]
 [0.04980132 0.9653948 ]
 [1.9934295  0.01223389]] 	 {'loss': [0.18477745354175568]}
700 [[0.90510756 0.03846198]
 [0.03247704 0.9632356 ]
 [1.9716064  0.00365077]] 	 {'loss': [0.14356191456317902]}
800 [[0.94708073 0.04317535]

Gradient descent found the correct filter. 

Again, that was possible because the system of equations for the filter components is over-determined, i.e., there are enough data to constrain the problem. Notice that I increased the length of the input sequence relative to the single-channel example in order to compensate for the fact that the filter now has a greater number of components. An alternative approach would have been to increase the number of examples fed to the training algorithm.

## Finding multiple 1D convolution filters

The final element of complexity is to consider multiple filters, each mapping the input sequence to a seperate output sequence. Gradient descent in Keras gets the job done here too, even though the system of equations is under-determined...

In [11]:
K.clear_session()

toyX = np.array([0, 3, 4, 5]).reshape(1,4,1)

# convolve two filters, [1, 2] and [3, 4], with the above sequence to yield:
toyY = np.array([[6, 12], [11, 25], [14, 32]]).reshape(1,3,2) 

toyModel = Sequential([
    Conv1D(filters=2, kernel_size=2, strides=1, padding='valid', use_bias=False, input_shape=(4,1), name='c1d')
])
toyModel.compile(optimizer=Adam(lr=5e-2), loss='mae')

print('shape of conv filter', toyModel.layers[0].get_weights()[0].shape)
print('conv weights')
print('------------')
for epoch in range(1000):
    history = toyModel.fit(toyX, toyY, verbose=0)
    if epoch%100 == 0:
        # kernel has dimension: kernel_size X number_input_channels X number_output_channels
        print("{:3d} {} \t {}".format(epoch, 
                                      toyModel.layers[0].get_weights()[0][:,0,:], 
                                      history.history))
print('prediction on training data:', toyModel.predict(toyX))

shape of conv filter (2, 1, 2)
conv weights
------------
  0 [[-0.6103726  -0.8287409 ]
 [-0.15447119 -0.89825535]] 	 {'loss': [20.767751693725586]}
100 [[1.0252689 3.6927207]
 [2.0115454 3.7466788]] 	 {'loss': [0.6135830879211426]}
200 [[0.9909381 2.9846911]
 [1.9965166 3.9881284]] 	 {'loss': [0.08037328720092773]}
300 [[0.9794749 2.9899173]
 [1.9887626 3.9881496]] 	 {'loss': [0.05458291247487068]}
400 [[0.9978222 3.0115416]
 [1.999234  4.0161667]] 	 {'loss': [0.03423619270324707]}
500 [[0.9978582 2.9963021]
 [1.9992551 3.9971995]] 	 {'loss': [0.011512279510498047]}
600 [[0.99785465 3.0019803 ]
 [1.9992567  4.0074735 ]] 	 {'loss': [0.016494354233145714]}
700 [[0.9978543 3.0014691]
 [1.9992565 3.9961095]] 	 {'loss': [0.01576344110071659]}
800 [[1.0117548 3.0033073]
 [2.0094244 4.0087605]] 	 {'loss': [0.032050926238298416]}
900 [[1.0040337 3.0018487]
 [1.9964517 3.995188 ]] 	 {'loss': [0.02338552474975586]}
prediction on training data: [[[ 6.011879 12.008711]
  [11.044497 24.983978]
  [